In [1]:
import spiketoolkit as st
import spikeextractors as se
import numpy as np
import spikemetrics.metrics as metrics
from spikemetrics.common import Epoch

16:13:38 [I] klustakwik KlustaKwik2 version 0.2.6
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/networkx-2.2-py3.7.egg/networkx/classes/reportviews.py:95: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable


In [2]:
# workshop_path = "/Users/colehurwitz/"
workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")


Could not load plane information. Assuming probe is in yz plane


In [3]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [15]:
st.sorters.KlustaSorter.default_params()

{'probe_file': None,
 'adjacency_radius': None,
 'threshold_strong_std_factor': 5,
 'threshold_weak_std_factor': 2,
 'detect_sign': -1,
 'extract_s_before': 16,
 'extract_s_after': 32,
 'n_features_per_channel': 3,
 'pca_n_waveforms_max': 10000,
 'num_starting_clusters': 50}

In [21]:
sorting = st.sorters.run_klusta(recording, threshold_weak_std_factor=1)

Klusta units are not unique! Using unique unit ids


In [22]:
num_spikes = 0
for unit_id in sorting.get_unit_ids():
    num_spikes += len(sorting.get_unit_spike_train(unit_id))
print(num_spikes)

5931


In [23]:
st.postprocessing.export_to_phy(recording, sorting, output_folder='phy_results')

In [6]:
metric_info = st.postprocessing.get_quality_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_waveforms=False, verbose=False)

In [7]:
spike_times = recording.frame_to_time(metric_info[0])
spike_clusters = metric_info[1].astype(int)
amplitudes = metric_info[2]
channel_map = metric_info[3]
pc_features = metric_info[4]
pc_feature_ind = metric_info[5]

In [8]:
params = {
          "isi_threshold" : 0.0015,
          "min_isi" : 0.000166,
          "num_channels_to_compare" : 13,
          "max_spikes_for_unit" : 500,
          "max_spikes_for_nn" : 10000,
          "n_neighbors" : 4,
          'n_silhouette' : 10000,
          "quality_metrics_output_file" : "metrics.csv",
          "drift_metrics_interval_s" : 51,
          "drift_metrics_min_spikes_per_interval" : 10
         }

In [10]:
epochs_dict = {
    'red': (300000, 960000),
    'blue': (0, 300000)
}

In [11]:
epochs = []
for key in list(epochs_dict.keys()):
    epoch_name = key
    epoch_start_frame = epochs_dict[key][0]
    epoch_end_frame = epochs_dict[key][1]
    epochs.append(Epoch(name=key, start_time=recording.frame_to_time(epoch_start_frame), end_time=recording.frame_to_time(epoch_end_frame)))
epochs = sorted(epochs, key=lambda epoch: epoch.start_time)

In [13]:
metrics_pd = metrics.calculate_metrics(spike_times=spike_times, spike_clusters=spike_clusters, amplitudes=amplitudes, channel_map=channel_map, pc_features=pc_features,\
                                       pc_feature_ind=pc_feature_ind, params=params, epochs = epochs)

Calculating isi violations
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating presence ratio
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating firing rate
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating amplitude cutoff
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating PC-based metrics
 ▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 18% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructur

 ▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 25% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 39% 

...

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░ 86% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating silhouette score
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating drift metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


/afs/inf.ed.ac.uk/user/s17/s1727843/spiketoolkit/spikemetrics/metrics.py:370: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(SS,0)


In [14]:
metrics_pd

,cluster_id,firing_rate,presence_ratio,isi_viol,amplitude_cutoff,isolation_distance,l_ratio,d_prime,nn_hit_rate,nn_miss_rate,silhouette_score,max_drift,cumulative_drift,epoch_name
0,0,2.351244,0.18,7.245925,0.0,1.248136e+02,3.952930e-08,2.879369,0.166667,0.004271,NaN,0.0,0.0,blue
1,1,14.855586,0.75,0.000000,0.0,1.087926e+02,7.436934e-04,4.976592,0.865707,0.010630,0.411916,0.0,0.0,blue
2,2,4.809362,0.32,0.000000,0.0,1.978181e+02,1.676600e-05,4.449334,0.903704,0.005946,0.223024,0.0,0.0,blue
3,3,1.709995,0.16,0.000000,0.0,7.071739e+14,NaN,4.806571,0.500000,0.001597,0.135786,0.0,0.0,blue
4,4,1.816870,0.15,0.000000,0.0,4.167633e+14,NaN,1.944586,0.372549,0.005147,0.065484,0.0,0.0,blue
5,5,2.137494,0.19,0.000000,0.0,8.049695e+14,NaN,3.967488,0.616667,0.003378,0.056754,0.0,0.0,blue
6,6,11.221845,0.63,0.000000,0.0,1.318327e+02,7.146336e-03,5.383017,0.860317,0.007635,0.104941,0.0,0.0,blue
7,7,3.847490,0.29,0.000000,0.0,2.323437e+02,6.672433e-08,7.839060,0.944444,0.003048,0.161904,0.0,0.0,blue
8,8,13.573089,0.76,0.000000,0.0,1.809619e+02,1.574361e-03,6.681727,0.900262,0.008673,0.157415,0.0,0.0,blue
9,9,2.778743,0.24,0.000000,0.0,9.885012e+14,NaN,5.986791,0.833333,0.002140,0.169084,0.0,0.0,blue
